# LangChain 기반 RAG 실습
### 블로그의 정보를 크롤링하여 챗봇 구현하기
- 외부 블로그 링크를 입력받는다
  - RAG internet source
- HTML 문서를 처리하는 방식을 구분한다
  - langchain.document_loader > **`WebBaseLoader`**
  - langchain_core.document > **`Document`**
- LLM
  - GPT > **`gpt-4o-mini`** 사용
- 답변 출력
  - 원하는 질문을 입력하여 답변을 얻어본다

## RAG(Retrieval Augmented Generation, 검색증강생성)

LLM의 출력을 최적화하여 응답을 생성하기 전에 **학습 데이터 소스 외부의 신뢰할 수 있는 지식 베이스를 참조하도록** 하는 프로세스   
LLM은 방대한 양의 데이터를 기반으로 학습되며 수십억 개의 매개 변수를 통해 질문에 대한 답변, 언어 번역, 문장 완성과 같은 작업에 대한 독창적인 결과를 생성   
<br>
👍🏻 RAG는 이미 강력한 LLM의 기능을 **특정 도메인이나 조직의 내부 지식 기반으로 확장**하므로 **모델을 다시 교육할 필요가 없음**   
✅ LLM 결과를 개선하여 다양한 상황에서 관련성, 정확성 및 유용성을 유지하기 위한 **비용 효율적**인 접근 방식

<br>

![RAG-Amazon](https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/jumpstart/jumpstart-fm-rag.jpg)   
출처: https://aws.amazon.com/ko/what-is/retrieval-augmented-generation/

## 어디에 쓰죠?
- 챗봇
- 기타 NLP 애플리케이션

## 구체적으로 왜 쓰죠?
RAG를 사용하는 이유는 **신뢰할 수 있는 지식 소스를 상호 참조**하여 다양한 상황에서 사용자 질문에 답변할 수 있는 봇 제작   
다만 LLM은 아래와 같은 **문제점**이 존재함
- LLM 응답에 대한 예측이 불가능
- 답변이 없을 경우 허위 정보를 제공하여 신뢰도가 낮음(출처도 신뢰할 수 없음)
- 유저가 구체적이고 최신 답변을 기대할 때 일반적인 정보를 제공
- 용어 혼동으로 인해 응답이 부정확함

👉🏻 즉, LLM은 **굉장히 열정적인 신입 사원** 느낌   
✅ RAG는 이러한 문제 중 일부를 해결하기 위한 방법
- LLM을 redirection하여 신뢰할 수 있는 사전 결정된 지식 출처에서 관련 정보를 검색
- 생성된 텍스트 출력을 보다 잘 제어할 수 있게 됨
- 유저는 LLM이 응답을 생성하는 방식에 대해 통찰력을 얻을 수 있음

In [ ]:
!pip install langchain_core langchain-chroma langchain-openai bs4

**WebBaseLoader vs Document**   
한글로 구성된 HTML을 크롤링하는 경우 `WebBaseLoader`를 사용하면 문장이 깨짐   
**`Document`**로 실행하여 한글 깨짐 방지

In [18]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document # WebBaseLoader => Document 수정
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

**OPEN AI api key 가져옴**

In [5]:
from google.colab import files
uploaded = files.upload()

Saving openAI-api-key.env to openAI-api-key.env


In [6]:
!pip install python-dotenv

In [8]:
from dotenv import load_dotenv
import os

load_dotenv("openAI-api-key.env")

api_key = os.getenv("OPENAI_API_KEY")

**GPT 4o-mini 모델 연결**

In [17]:
# 답변의 일관성을 위해 temperature 0 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=api_key)

**크롤링**

In [13]:
import requests
import json

# 크롤링 대상 URL
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"

# HTML 요청
res = requests.get(url)

# 인코딩 자동 감지 (한글 깨짐 방지)
encoding = res.encoding
if encoding is None:
    encoding = 'utf-8'
res.encoding = encoding

# BeautifulSoup으로 HTML 파싱
soup = bs4.BeautifulSoup(res.text, "html.parser")

# 크롤링 데이터 추출
content = soup.get_text()

# LangChain의 Document 객체로 매핑
docs = [Document(page_content=content)]

In [14]:
# dict() deprecated => model_dump()
print(json.dumps(docs[0].model_dump(), indent=2, ensure_ascii=False))

{
  "id": null,
  "metadata": {},
  "page_content": "스파르타코딩클럽 | 블로그포인트로딩중쿠폰내 강의실국비 신청 내역수강권증명서숙제 피드백계정로그아웃 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  817·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆 대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있

**문서 분할**   
- RAG는 전체 문서를 그대로 사용하지 않음
- 검색 효율을 높이기 위해 일정 길이로 나눈 **청크**를 사용
- `RecursiveCharacterTextSplitter`
  - 문장을 최대한 자연스럽게 나누는 분할 방식

In [15]:
# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,   # 한 청크의 최대 길이(문자 수)
    chunk_overlap=50  # 청크 간 겹치는 길이(context 유지)
)

# 청크 단위로 문서를 분할
splits = text_splitter.split_documents(docs)

# 확인해보기
print(f"🔹 분할된 문서 수: {len(splits)}")
print(f"🔹 첫 번째 조각:\n{splits[0].page_content[:300]}")

🔹 분할된 문서 수: 14
🔹 첫 번째 조각:
스파르타코딩클럽 | 블로그포인트로딩중쿠폰내 강의실국비 신청 내역수강권증명서숙제 피드백계정로그아웃 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  817·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠


**분할된 청크를 Vector 저장소에 삽입**   
- RAG에서 **정보 검색**을 가능하게 만드는 과정

<br>

**Vector store 구성**   
- Chroma + OpenAI 임베딩

In [16]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

### ✅ RAG 구성
**retriever + GPT 연결**

In [20]:
# 벡터스토어 -> retriever 변환
retriever = vectorstore.as_retriever()

# RAG 체인 구성
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True # 출처 문서도 반환
)

### 🤔 질문 입력 & 응답 출력

In [30]:
# 질문
query = "ALL-in 코딩 공모전 수상작들을 요약해줘."

# RAG 실행
result = chain(query)

# 결과
print("✅ 요약 결과")
print(result['result'])

# 참조 문서

✅ 요약 결과
ALL-in 코딩 공모전에서 수상한 작품들은 다음과 같습니다:

1. **대상: Lexi Note** - 언어공부 필기 웹 서비스로, 어문학을 전공하는 대학생들이 제작했습니다. 이 서비스는 언어 학습을 지원하는 기능을 제공합니다.

2. **우수상: 에코 클래스룸** - 수업 실시간 소통 서비스로, 학생들이 수업 후 질문할 수 있는 환경을 제공하여 교수와 학생 간의 소통을 개선하는 데 초점을 맞추고 있습니다.

이 외에도 다양한 혁신적인 아이디어와 프로젝트들이 공모전에서 소개되었습니다. 각 프로젝트는 대학생들이 캠퍼스에서 겪는 문제를 해결하기 위한 아이디어에서 출발했습니다.


**(추가) GPT 응답 Markdown**

In [31]:
from IPython.display import Markdown, display

In [32]:
markdown_text = result["result"]  # GPT 요약 결과가 마크다운이라면 그대로 출력
display(Markdown(markdown_text))

ALL-in 코딩 공모전에서 수상한 작품들은 다음과 같습니다:

1. **대상: Lexi Note** - 언어공부 필기 웹 서비스로, 어문학을 전공하는 대학생들이 제작했습니다. 이 서비스는 언어 학습을 지원하는 기능을 제공합니다.

2. **우수상: 에코 클래스룸** - 수업 실시간 소통 서비스로, 학생들이 수업 후 질문할 수 있는 환경을 제공하여 교수와 학생 간의 소통을 개선하는 데 초점을 맞추고 있습니다.

이 외에도 다양한 혁신적인 아이디어와 프로젝트들이 공모전에서 소개되었습니다. 각 프로젝트는 대학생들이 캠퍼스에서 겪는 문제를 해결하기 위한 아이디어에서 출발했습니다.